First goal is to find an endpoint for Form 4's

https://www.sec.gov/developer

https://www.sec.gov/os/accessing-edgar-data

In [15]:
import requests

run_once_override = False

Fetch the list of CIKs from here https://www.sec.gov/Archives/edgar/cik-lookup-data.txt

In [20]:
headers = {
    'User-Agent': 'Personal User khalidelassaad@gmail.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov'
}

def get_url_to_outfilename(url, outfilename, headers=headers, run_once_override=run_once_override):
    if not run_once_override:
        return 0
    response = requests.get(url, headers=headers)
    print("Status Code:", response.status_code)
    with open(outfilename, "w", encoding="utf-8") as outfile:
        outfile.write(response.text)
    return response.status_code

In [22]:
get_url_to_outfilename(
    "https://www.sec.gov/Archives/edgar/cik-lookup-data.txt",
    "cik_list.txt"
)

0

In [19]:
not run_once_override

True